In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DoubleType, IntegerType

In [0]:
checkpoint_path = "/Volumes/workspace/silver/checkpoint"

In [0]:
df = spark.readStream.format("delta").table("workspace.bronze.orders_stream")

In [0]:
# select only related columns
unwanted_columns = ['kafka_timestamp', 'kafka_partition', 'kafka_offset', 'timestamp']
df = df.select([col for col in df.columns if col not in unwanted_columns])

In [0]:
# trimming all spaces
string_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
for col in string_cols:
    df = df.withColumn(col, F.trim(F.col(col)))

In [0]:
#convert date columns
df = (
    df.withColumn("sls_order_dt",
      F.when((F.col("sls_order_dt") == 0) | (F.length(F.col("sls_order_dt")) != 8), None)
       .otherwise(F.to_date(F.col("sls_order_dt").cast("string"), "yyyyMMdd"))
    ).withColumn("sls_ship_dt",
      F.when((F.col("sls_ship_dt") == 0) | (F.length(F.col("sls_ship_dt")) != 8), None)
       .otherwise(F.to_date(F.col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    ).withColumn("sls_due_dt",
      F.when((F.col("sls_due_dt") == 0) | (F.length(F.col("sls_due_dt")) != 8), None)
       .otherwise(F.to_date(F.col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )
)

In [0]:
# sls_price * sls_quantity = sls_sales

df = df.withColumn(
    "sls_price",
    F.when((F.col("sls_price").isNull()) | (F.col("sls_price") <=0),
           F.when(F.col("sls_price") != 0, F.col("sls_sales") / F.col("sls_quantity")).otherwise(None) 
        ).otherwise(F.col("sls_price"))
)

In [0]:
#rename columns

_RENAME_MAP = {
    "Kafka_key": "order_number",
    "sls_prd_key": "product_number",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}
for old_name, new_name in _RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df = df.select(
    F.col("order_number"), F.col("product_number"), F.col("customer_id").cast(IntegerType()), F.col("order_date"), F.col("ship_date"), F.col("due_date"), F.col("sales_amount").cast(DoubleType()), F.col("quantity").cast(IntegerType()), F.col("price").cast(DoubleType())
    )

In [0]:
query = df.writeStream.outputMode("append").format("delta") .option("checkpointLocation", checkpoint_path).option("mergeSchema", "true").trigger(availableNow=True).table("workspace.silver.kafka_orders")

query.awaitTermination()